# En este Notebook aplicaremos la discretización, por intervalos, cuantil, por clustering,  en intervalos arbitrarios y discretizacion supervisada.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
# Para discretización
from sklearn.preprocessing import KBinsDiscretizer

In [ ]:
# Cargando datos Boston House
data = pd.read_csv('../../datasets/raw/boston.csv')
data.head()

In [ ]:
# Separando los datos
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('MEDV', axis=1), data['MEDV'], test_size=0.3, random_state=0)

X_train.shape, X_test.shape

In [ ]:
# Identificamos y graficamos las variables que deseamos discretizar
feature_to_discretize=['LSTAT', 'DIS', 'RM']

# Exploremos la distribución 
X_train[feature_to_discretize].hist(bins=30, figsize=(10,6))
plt.show()

### Discretización igual intervalo

In [ ]:
# Configuremos y entrenemos un discretizador por intervalos de 10 bins

disc = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy= 'uniform', random_state=1)

disc.fit(X_train[feature_to_discretize])

# Los límites de los intervalos son almacenados en bin_edges_attribute
disc.bin_edges_

In [ ]:
# Tranformemos el set de entrenamiento. Recordar que Sklearn regresa Numpy arrays
train_t = disc.transform(X_train[feature_to_discretize])

# Transformemos el array en dataframe
train_t = pd.DataFrame(train_t, columns = feature_to_discretize)

train_t.head()

In [ ]:
# Grafiquemos y comparemos
X_train[feature_to_discretize].hist(bins=30, figsize=(10,6))
plt.show()

train_t

train_t.hist(bins=30, figsize=(10,6))
plt.show()


In [ ]:
# El mismo procedimiento para el conjunto de prueba

test_t = disc.transform(X_test[feature_to_discretize])

test_t = pd.DataFrame(test_t, columns = feature_to_discretize)
test_t.head(100)

### Discretización igual frecuencia

In [ ]:
# Configuremos y entrenemos un discretizador basado en cuantiles con 10 bins
disc = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile',random_state=1)

disc.fit(X_train[feature_to_discretize])

# Los limites de los intervalos son almacenados en un atributo transformado:
disc.bin_edges_

In [ ]:
# Tranformemos el set de entrenamiento. Recordar que Sklearn regresa Numpy arrays
train_t = disc.transform(X_train[feature_to_discretize])

train_t = pd.DataFrame(train_t, columns = feature_to_discretize)

train_t.head()

In [ ]:
# Grafiquemos y comparemos
X_train[feature_to_discretize].hist(bins=30, figsize=(10,6))
plt.show()

train_t

train_t.hist(bins=30, figsize=(10,6))
plt.show()

In [ ]:
# Sobre el conjunto de entrenamiento
test_t = disc.transform(X_test[feature_to_discretize])
test_t = pd.DataFrame(test_t, columns = feature_to_discretize)
test_t.head()

### Discretizacion utilizando k-means

In [ ]:
# Configuremos y entrenemos un discretizador k-means
disc = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='kmeans', random_state=1)
disc.fit(X_train[feature_to_discretize])

# Inspeccionamos las fronteras de los bins:
disc.bin_edges_

In [ ]:
# Apliquemos la transformación al conjunto de entrenamiento
train_t = disc.transform(X_train[feature_to_discretize])
train_t = pd.DataFrame(train_t, columns = feature_to_discretize)

print(train_t.head(20))


In [ ]:
# Grafiquemos y comparemos
X_train[feature_to_discretize].hist(bins=30, figsize=(10,6))
plt.show()

train_t

train_t.hist(bins=30, figsize=(10,6))
plt.show()

In [ ]:
# Sobre el conjunto de prueba

test_t = disc.transform(X_test[feature_to_discretize])
test_t = pd.DataFrame(test_t, columns = feature_to_discretize)
test_t.head()

In [ ]:
#Grafiquemos y comparemos en e cto. de prueba
X_test[feature_to_discretize].hist(bins=30, figsize=(10,6))
plt.show()

test_t

test_t.hist(bins=30, figsize=(10,6))
plt.show()

### Discretización por intervalos arbitrarios

Primero necesitamos determinar los límites de los intervalos

In [ ]:
# Grafiquemos el histograma para variable LSTAT

data['LSTAT'].hist(bins=30)

In [ ]:
# Definimos las fronteras del bin
intervals = [0, 10, 20, 30, np.Inf]

# Etiquetas de los bins
labels = ['0-10', '10-20', '20-30', '>30']

In [ ]:
# Discretizemos LSTAT

# Version discretizada con strings
data['lstat_labels'] = pd.cut(
    data['LSTAT'], bins=intervals, labels=labels, include_lowest=True)

# Versión discretizada con los limites de los intervalos
data['lstat_intervals'] = pd.cut(
    data['LSTAT'], bins=intervals, labels=None, include_lowest=True)

# Mostramos la original y ambas versiones discretizadas de la variable
print(data[['LSTAT', 'lstat_labels', 'lstat_intervals']].head(30))

In [ ]:
# Revisemos el número de observaciones por bin

data['lstat_intervals'].value_counts()

In [ ]:
fig= data['lstat_intervals'].value_counts().plot.bar()

### Discretización supervisada utilizando un Entropy based discretizer

In [ ]:
# Utlizamos un nuevo conjunto de datos de UCI disponible en sklearn
from sklearn.datasets import load_wine

data = load_wine()
df = pd.DataFrame(data.data, columns=data.feature_names)
df.head()


In [ ]:
df.describe()

In [ ]:
#Elegimos la variable a discretizar y el target multiclase.

variable = "magnesium"
x = df[variable].values
y = data.target


In [ ]:
# Importamos e instanciamos un objeto de MDLP de la librería optbinning. 
from optbinning import MDLP

mdlp = MDLP()
mdlp.fit(x,y)
splits=mdlp.splits


In [ ]:
finalSplits=splits.tolist()
finalSplits.insert(0,0)
finalSplits.append(np.inf)

print(finalSplits)

In [ ]:
# Versión discretizada con los limites de los intervalos
df['magnesium_intervals'] = pd.cut(
    df['magnesium'], bins=finalSplits, labels=None, include_lowest=True)

# Mostramos la original y ambas versiones discretizadas de la variable
print(df[['magnesium','magnesium_intervals']].head(30))


In [ ]:
#Grafiquemos y comparemos
df['magnesium'].hist(bins=30)
plt.show()

In [ ]:
fig= df['magnesium_intervals'].value_counts().plot.bar()

### Discretización supervisada utilizando arboles de desicion

Se deje como ejercicio